In [23]:
#Importing relevant libraries 
import pandas as pd
import numpy as np 
import requests 
import mysql.connector as mysql

In [3]:
##Extracting data from facebook api 
#Setting up the url 
url = "https://graph.facebook.com/v12.0/me?access_token="
token = 'EAAEHNsZAK8n0BACIuGMWGuzvQRcZAmx39NU40MWnGlnqN70WF9bGhKrkdLWx1ZCPi0JqsdJa4Ql7Nx2HFXuIblptHTeoLyixkJgLgiQfCCirZCK7HlWhK4bZBvphXHg4maC9QBrZAQqqpQZAZCQLRZBgWWyJr4jObWa1FS605a7L6sZCZAhDPtabusvx0U1TbF5v37dTEkTu8STfpmJHvseJ3Qvaj6ob6NGNTYZC6HuDRnFHtHyAsHMruKww'
full_url = url + token 
myuser_info = requests.get(full_url)
#Exracting my user info
myuser_info = requests.get(full_url).json()
print(myuser_info)

{'name': 'Edson Kambeu', 'id': '10226451930275156'}


In [7]:
##Extracting information about page posts 
url_posts = "https://graph.facebook.com/v12.0/me/posts?fields=id,created_time,message,shares,object_id,time_visibility,type,updated_time&limit=1000&access_token=" + token
#Extracting posts data
myuser_posts = requests.get(url_posts).json()
#Viewing the first posts 
print(myuser_posts['data'][0:4])

[{'id': '10226451930275156_10226602820727323', 'created_time': '2022-01-24T17:07:07+0000', 'message': 'The Yellow we love💛💛💛💛🤗🤗', 'object_id': '10226602820567319', 'type': 'photo', 'updated_time': '2022-01-24T17:07:07+0000'}, {'id': '10226451930275156_10226525280268860', 'created_time': '2022-01-10T14:55:15+0000', 'message': 'Painful to concede at the end..but well played', 'type': 'status', 'updated_time': '2022-01-10T14:55:15+0000'}, {'id': '10226451930275156_10226524072998679', 'created_time': '2022-01-10T10:13:57+0000', 'object_id': '2190634834408352', 'type': 'photo', 'updated_time': '2022-01-10T10:13:57+0000'}, {'id': '10226451930275156_10226455950215652', 'created_time': '2021-12-28T06:24:01+0000', 'message': 'This game was lit. Newcastle played like they were possesed. De Gea putting in some stunning saves🙌', 'object_id': '436198218210685', 'type': 'video', 'updated_time': '2021-12-28T06:24:01+0000'}]


In [8]:
##Adding missing keys in an object-using object index 4 
object_fields = ['id', 'created_time', 'message', 'shares', 'object_id', 'time_visibility', 'type', 'updated_time']
my_dict = myuser_posts['data'][4]
for object_field in object_fields:
    if object_field not in my_dict.keys():
        my_dict[object_field] = None
print(my_dict)


{'id': '10226451930275156_10226425344370525', 'created_time': '2021-12-22T10:43:23+0000', 'object_id': '2176325309172638', 'type': 'photo', 'updated_time': '2021-12-22T10:43:23+0000', 'message': None, 'shares': None, 'time_visibility': None}


In [9]:
#Extracting data values from each key
post_id = my_dict['id']
post_datetime = my_dict['created_time']
def shares_value(): 
    if my_dict['shares'] != None:
        value = my_dict['shares']['count']
    else:
        value = my_dict['shares']
    return value
post_shares = shares_value()
post_object_id = my_dict['object_id']
post_object_type = my_dict['type']
post_time_visibility = my_dict['time_visibility']
last_updated_time = my_dict['updated_time']

In [10]:
#Creating a function that extracts data from an object 
def get_object_data(object_index):
    object_fields = ['id', 'created_time', 'message', 'shares', 'object_id', 'time_visibility', 'type', 'updated_time']
    my_dict = myuser_posts['data'][object_index]
    for object_field in object_fields:
        if object_field not in my_dict.keys():
            my_dict[object_field] = None
#Extracting data values from each key in an object 
    post_id = my_dict['id']
    post_datetime = my_dict['created_time']
    post_message = my_dict['message']
    def shares_value(): 
        if my_dict['shares'] != None:
            value = my_dict['shares']['count']
        else:
            value = my_dict['shares']   
        return value
    post_shares = shares_value()
    post_object_id = my_dict['object_id']
    post_object_type = my_dict['type']
    post_time_visibility = my_dict['time_visibility']
    last_updated_time = my_dict['updated_time']
#Creating a new dictionary
    new_dict = {
        object_index: {'post_id': post_id,
              'post_datetime': post_datetime,
              'post_message': post_message,
              'post_shares' : post_shares,
              'post_object_id': post_object_id,
              'post_object_type': post_object_type,
              'post_time_visibility': post_time_visibility,
              'last_updated_time': last_updated_time }
    }
    return new_dict

In [11]:
#Testing the function-getting the data for object(index =28) 
get_object_data(28)

{28: {'post_id': '10226451930275156_10224479575447518',
  'post_datetime': '2021-02-19T11:12:19+0000',
  'post_message': None,
  'post_shares': None,
  'post_object_id': '10224479575247513',
  'post_object_type': 'photo',
  'post_time_visibility': None,
  'last_updated_time': '2021-02-19T11:12:19+0000'}}

In [12]:
#Get data for all the objects and store them in a pandas dataframe
def get_all_data():
        my_newest_dict = { }
        for object_index in range (0,len(myuser_posts['data'])):
                data = get_object_data(object_index)
                my_newest_dict.update(data)
                data_df = pd.DataFrame.from_dict(my_newest_dict, orient= 'index')
                
        return data_df    

In [13]:
#Printing the pandas dataframe
my_df = get_all_data()

#Viewing the first 6 rows of my pandas dataframe
my_df.head()

,post_id,post_datetime,post_message,post_shares,post_object_id,post_object_type,post_time_visibility,last_updated_time
0,10226451930275156_10226602820727323,2022-01-24T17:07:07+0000,The Yellow we love💛💛💛💛🤗🤗,NaN,10226602820567319,photo,None,2022-01-24T17:07:07+0000
1,10226451930275156_10226525280268860,2022-01-10T14:55:15+0000,Painful to concede at the end..but well played,NaN,None,status,None,2022-01-10T14:55:15+0000
2,10226451930275156_10226524072998679,2022-01-10T10:13:57+0000,None,NaN,2190634834408352,photo,None,2022-01-10T10:13:57+0000
3,10226451930275156_10226455950215652,2021-12-28T06:24:01+0000,This game was lit. Newcastle played like they ...,NaN,436198218210685,video,None,2021-12-28T06:24:01+0000
4,10226451930275156_10226425344370525,2021-12-22T10:43:23+0000,None,NaN,2176325309172638,photo,None,2021-12-22T10:43:23+0000


In [14]:
#checking the data type 
print(my_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 249 entries, 0 to 248
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   post_id               249 non-null    object 
 1   post_datetime         249 non-null    object 
 2   post_message          218 non-null    object 
 3   post_shares           2 non-null      float64
 4   post_object_id        90 non-null     object 
 5   post_object_type      249 non-null    object 
 6   post_time_visibility  0 non-null      object 
 7   last_updated_time     249 non-null    object 
dtypes: float64(1), object(7)
memory usage: 17.5+ KB
None


In [250]:
#Saving the data frame as a csv file 
#my_df.to_csv("facebook_posts.csv", index=False)

In [15]:
##Converting post_shares nan values into none as required in sql 
#Converting post_shares nan to none 
my_df['post_shares'] = my_df['post_shares'].astype('object').where(my_df['post_shares'].notna(), None )
#Checking whether nan has been converted to none
print(my_df['post_shares'].head())

0    None
1    None
2    None
3    None
4    None
Name: post_shares, dtype: object


In [16]:
##Converting post_datetime date into a form accceptable in SQL
#post_datetime
for index in range(0, len(my_df['post_datetime'])):
    my_df['post_datetime'][index] = my_df['post_datetime'][index][:-5].replace('T', ' ')
#Checking whether post_date_time has been converted 
my_df['post_datetime'].head()


0    2022-01-24 17:07:07
1    2022-01-10 14:55:15
2    2022-01-10 10:13:57
3    2021-12-28 06:24:01
4    2021-12-22 10:43:23
Name: post_datetime, dtype: object

In [17]:
##Converting last_updated_time date into a form accceptable in SQL
#last_updated_time
for index in range(0, len(my_df['post_datetime'])):
    my_df['last_updated_time'][index] = my_df['last_updated_time'][index][:-5].replace('T', ' ')
#Check whether last_updated_time has been converted 
my_df['last_updated_time'].head()

0    2022-01-24 17:07:07
1    2022-01-10 14:55:15
2    2022-01-10 10:13:57
3    2021-12-28 06:24:01
4    2021-12-22 10:43:23
Name: last_updated_time, dtype: object

In [18]:
#Modifying the post ID to remain with the post identifier
for index in range(0, len(my_df['post_id'])):
    my_df['post_id'][index] = my_df['post_id'][index][18:]
#Checking if post id has been modified 
print(my_df['post_id'].head())


0    10226602820727323
1    10226525280268860
2    10226524072998679
3    10226455950215652
4    10226425344370525
Name: post_id, dtype: object


In [21]:
#A function to convert dataframe to a list of tuples
def df_to_list_tuple (): 
    x= []
    for row in my_df.itertuples(index=False, name= None):
        x.append(row)
    return(x)

In [22]:
#Getting the data as a tuple
my_tuple = df_to_list_tuple()
#Checking if its a list 
print(type(my_tuple))
#Checking if list has tuples inside 
print(type(my_tuple[0]))

<class 'list'>
<class 'tuple'>


In [25]:
##Inserting the data into mysql database
#Establishing a connection  
try:
    conn = mysql.connect(user = 'root',
                         password = 'jayden@2022',
                         host = '127.0.0.1',
                         database = 'facebook')

except mysql.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print('You user name or password is incorrect')
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print('Database does not exist')
    else :
        print(err)

#Creating a cursor object
cursor = conn.cursor()

#Preparing queryto insert data into SQL
sql = """INSERT INTO facebook_posts (post_id, post_datetime, post_message, post_shares, post_object_id, post_object_type, post_time_visibility, last_updated_time)
VALUES (%s, %s, %s,%s ,%s , %s,%s, %s)"""

records = my_tuple
try:
    #Executing the sql statement
    cursor.executemany(sql, records)
    
    #Commit changes to the database
    conn.commit()
    print('Data sucessfully inserted into mysql database')
except  mysql.Error as err:
    print('Somthing went wrong whilst inserting the data into mysql')
    print(err)
    print("Error code:", err.errno)
    print("SQL State:", err.sqlstate)
    print("Message:", err.msg)

#close the connection 
conn.close()

Data sucessfully inserted into mysql database
